In [78]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from pathlib import Path
from numpy import mean
from numpy import std
from numpy import cov
from scipy.stats import pearsonr
import requests
import urllib.request
import json
import pprint
from dotenv import load_dotenv

%matplotlib inline

In [92]:
load_dotenv()
access_token = os.getenv("CQ_API_KEY")
headers = { 'Authorization': 'Bearer ' + access_token }



In [90]:
URL = "https://api.cryptoquant.com/v1/btc/exchange-flows/netflow?exchange=all_exchange&window=day&from=20150101&limit=100"
response = requests.get(URL, headers=headers)

netflow_data = response.json()
netflow_df = pd.DataFrame(data=netflow_data['result']['data'])
netflow_df['date'] = pd.to_datetime(netflow_df['date'])
netflow_df['netflow_total'] = netflow_df['netflow_total'].astype(str).astype('float64')
netflow_df = netflow_df.dropna()








In [91]:
URL2 = "https://api.cryptoquant.com/v1/btc/market-data/price-usd?window=day&from=20150101&limit=100"
response2 = requests.get(URL2, headers=headers)

price_data = response2.json()

price_df = pd.DataFrame(data=price_data['result']['data'])
price_df = price_df.drop(['price_usd_high','price_usd_low','price_usd_open'], axis = 1)
price_df['date'] = pd.to_datetime(price_df['date'])
price_df['price_usd_close'] = price_df['price_usd_close'].astype(str).astype('float64')
price_df = price_df.dropna()




In [82]:
covariance = cov(netflow_df['netflow_total'],price_df['price_usd_close'])
corr, _ = pearsonr(netflow_df['netflow_total'], price_df['price_usd_close'])
print('Pearsons correlation: %.3f' % corr)


Pearsons correlation: 0.213


In [85]:
def pearsons_coefficient(series1,series2):
    
  #Returns pearsons coefficient (int) between 2 dataframes columns.

       # Parameters: 
            ## series1 (object): First series to compare
            ## series2 (object): Second series to compare
            ## df1_column (object): Series from df1 we wish to compare
            ## df2_column (object): Series from df2 we wish to compare

    

    covariance = cov(series1,series2)
    corr, _ = pearsonr(series1,series2)
    print('Pearsons correlation: %.3f' % corr)
    return corr



In [86]:
test_df = pearsons_coefficient(netflow_df['netflow_total'],price_df['price_usd_close'])

Pearsons correlation: 0.213


In [105]:
def get_onchain_data(ticker,fromdate,limit):
    '''
    Returns on-chain data (Netflow)n in new df
        Parameters:
            ticker (string) = Ticker for Desired Coin
            fromdate (string) = Starting date of period desired
            limit (int) = number of data points     
    '''
    #Initialize dotenv function, pull API key, build URL
    headers = {'Authorization': 'Bearer ' + access_token}
    URL = f'https://api.cryptoquant.com/v1/{ticker}/exchange-flows/netflow?exchange=all_exchange&window=day&from={fromdate}&limit={limit}'

    # Initialize Dictionary
    response = requests.get(URL, headers=headers)
    
    netflow_data = response.json()

    #Transform to Dataframe and adjust data type
    netflow_df = pd.DataFrame(data=netflow_data['result']['data'])
    netflow_df['date'] = pd.to_datetime(netflow_df['date'])
    netflow_df['netflow_total'] = netflow_df['netflow_total'].astype(str).astype('float64')
    return netflow_df


In [107]:
#test1_df = get_onchain_data("btc","20190101","100")

#print(test1_df)

In [144]:
URL3 = "https://api.cryptoquant.com/v1/btc/market-data/capitalization?window=day&from=20191001&limit=100"
response3 = requests.get(URL3, headers=headers)

mvrv_data = response3.json()

mvrv_data = pd.DataFrame(data=mvrv_data['result']['data'])
m_cap_stdv = np.std(mvrv_data['market_cap'])

mvrv_data['mvrv_score'] = (mvrv_data['market_cap']/mvrv_data['realized_cap'])
mvrv_data = mvrv_data.drop(['average_cap', 'delta_cap', 'market_cap','realized_cap','thermo_cap'], axis = 1)


mvrv_data.head()



,date,mvrv_score
0,2021-01-21,2.429670
1,2021-01-20,2.796105
2,2021-01-19,2.833096
3,2021-01-18,2.905155
4,2021-01-17,2.855650


In [145]:
def get_mvrv_data(ticker,fromdate,limit):
    '''
    Returns marketcap data in new df
        Parameters:
            ticker (string) = Ticker for Desired Coin
            fromdate (string) = Starting date of period desired
            limit (int) = number of data points     
    '''
    #Initialize dotenv function, pull API key, build URL
    headers = {'Authorization': 'Bearer ' + access_token}
    URL3 = f'https://api.cryptoquant.com/v1/{ticker}/market-data/capitalization?window=day&from={fromdate}&limit={limit}'
            
    # Initialize Dictionary
    response3 = requests.get(URL3, headers=headers)
    
    mvrv_data = response3.json()

    mvrv_data = pd.DataFrame(data=mvrv_data['result']['data'])
    m_cap_stdv = np.std(mvrv_data['market_cap'])
    mvrv_data['mvrv_score'] = (mvrv_data['market_cap']/mvrv_data['realized_cap'])
    mvrv_data = mvrv_data.drop(['average_cap', 'delta_cap', 'market_cap','realized_cap','thermo_cap'], axis = 1)
    
    
    return mvrv_data



In [149]:
test1_df = get_mvrv_data("btc","20190101","100000")

test1_df



,date,mvrv_score
0,2021-01-21,2.429670
1,2021-01-20,2.796105
2,2021-01-19,2.833096
3,2021-01-18,2.905155
4,2021-01-17,2.855650
5,2021-01-16,2.874996
6,2021-01-15,2.946355
7,2021-01-14,3.154689
8,2021-01-13,3.041982
9,2021-01-12,2.783554
